# IDR0021 - ROIs

A simple example which reads the 'batch_roi_export' table, performs some basic statistics on it and creates a plot (which should be similar to [Figure 1](https://www.nature.com/articles/ncb2591/figures/1) in the paper ['Subdiffraction imaging of centrosomes reveals higher-order organizational features of pericentriolar material'](https://doi.org/10.1038/ncb2591)). The plot and results are then uploaded to the server and attached to the project.

In [ ]:
options(warn=-1) # Disable warnings to suppress unnecessary output

## Load the romero.gateway library and connect to the server

In [ ]:
library(romero.gateway)

server <- OMEROServer(host = 'outreach.openmicroscopy.org', port = 4064L, username = 'username', password = 'password')
server <- connect(server)

## Load the project and get the id of the file annotation

In [ ]:
projectName <- 'idr0021'
tableName <- 'batch_roi_export'

searchResult <- searchFor(server, Project, 'Name', projectName)
project <- searchResult[[1]]
paste('Project:', project@dataobject$getName(), 'loaded')

annos <- getAnnotations(server, 'ProjectData', getOMEROID(project), nameFilter = tableName)
annotationFileID = as.integer(annos$FileID)
paste('Annotion file id:', annotationFileID)

## Check if we can load the file as R dataframe

In [ ]:
describeDataframe(project, annotationFileID)

## Load the relevant data (Image and max_points)

In [ ]:
df <- loadDataframe(project, annotationFileID, columns=c(1, 8))

## Prepare the data

### Remove the 'omero.gateway...' from the Image field to get the image id

In [ ]:
df$Image <- gsub("omero\\.gateway\\.model\\.ImageData \\(id=", "", df$Image)
df$Image <- gsub("\\)", "", df$Image)

### Estimate the toroid/ring diameter from the max_points (~ area)

In [ ]:
df$diameter <- 2 * sqrt(df$max_points / pi)

### Iterate through the datasets to get the image - protein (dataset name) relation and add it to the dataframe

In [ ]:
datasetImageMap <- data.frame()
datasets <- getDatasets(project)
for (dataset in datasets) {
  images <- getImages(dataset)
  name <- as.character(dataset@dataobject$getName())
  for(image in images) {
    id <- getOMEROID(image)
    datasetImageMap<-rbind(datasetImageMap, data.frame(Dataset=name, Image=id))
  }
}
df <- merge(df, datasetImageMap)

### Combine the datasets 'CEP120/20111106', 'CEP120/20111209' to 'CEP120 (see paper)

In [ ]:
df$Dataset <- as.character(df$Dataset)
df[ df == 'CEP120/20111106' ] <- 'CEP120'
df[ df == 'CEP120/20111209' ] <- 'CEP120'
df$Dataset <- as.factor(df$Dataset)

The dataframe looks like this now:

In [ ]:
df

## Plot

### Order the data from lowest to highest mean diameter

In [ ]:
ag <-aggregate(df$diameter ~ df$Dataset, df, mean)
orderedDatasets <- factor(df$Dataset, levels=ag[order(ag$`df$diameter`), 'df$Dataset'])

### Create the plot

In [ ]:
plot(df$diameter ~ orderedDatasets, ylab='Estimated toroid/ring diameter', xlab="Protein", cex.axis=0.5)

## Is there a significant difference between the proteins?

One-way analysis of variance:

In [ ]:
fit <- aov(df$diameter ~ df$Dataset)
summary(fit)

**There is!**

Two-sample Wilcoxon test ('Mann-Whitney') of all pairwise combinations:

In [ ]:
combins <- combn(levels(df$Dataset), 2)
params_list <- split(as.vector(combins), rep(1:ncol(combins), each = nrow(combins)))
testResults <- data.frame()
for (param in params_list) {
  testdf <- subset(df, df$Dataset %in% param)
  pval <- wilcox.test(formula = diameter ~ Dataset, data = testdf)$p.value
  testResults<-rbind(testResults, data.frame(Protein_1=param[1], Protein_2=param[2], p_value=pval))
}
testResults

## Attach the results to OMERO project

Attach the plot:

In [ ]:
tmpfile <- "/tmp/Toroid_diameter_By_Protein.png"
png(tmpfile, width = 4, height= 2.5, units = "in", res = 300, pointsize = 6)
plot(df$diameter ~ orderedDatasets, ylab='Estimated toroid/ring diameter', xlab="Protein", cex.axis=0.5)
dev.off()
invisible(attachFile(project, tmpfile)) # Wrapped in invisible(...) only to suppress unnecessary output

Attach the dataframe with the p-values from the Mann-Whitney test:

In [ ]:
# As OMERO.table (HDF)
invisible(attachDataframe(project, testResults, "Mann-Whitney-pValues"))

# As CSV file:
tmpfile <- "/tmp/Mann-Whitney-pValues.csv"
write.csv(testResults, file = tmpfile)
invisible(attachFile(project, tmpfile)) 

## Disconnect from the server again

In [ ]:
invisible(disconnect(server))